In [1]:
import pandas as pd
import re
import numpy as np
import os

In [2]:
files = [f'../scraped-data-2024/c17/{f}' for f in os.listdir('../scraped-data-2024/c17')]
files

['../scraped-data-2024/c17/World Championship 2015 - Ilca 6.xlsx',
 '../scraped-data-2024/c17/World Championship 2015 - Ilca 7.xlsx',
 '../scraped-data-2024/c17/World Championship 2016 - Ilca 6.xlsx',
 '../scraped-data-2024/c17/World Championship 2016 - Ilca 7.xlsx',
 '../scraped-data-2024/c17/World Championship 2017 - Ilca 6.xlsx',
 '../scraped-data-2024/c17/World Championship 2017 - Ilca 7.xlsx',
 '../scraped-data-2024/c17/World Championship 2019 - Ilca 6.xlsx',
 '../scraped-data-2024/c17/World Championship 2019 - Ilca 7.xlsx',
 '../scraped-data-2024/c17/World Championship 2020 - Ilca 6.xlsx',
 '../scraped-data-2024/c17/World Championship 2020 - Ilca 7.xlsx']

In [3]:
def formata_pontuacao(valor:str):
    valor_str = str(valor)
    numero_match = re.search(r'-?\d+(\.\d+)?', valor_str)
    numero = float(numero_match.group()) if numero_match else np.nan
    
    # Verificar se contém parênteses
    parenteses = 1 if re.search(r'\(.*?\)', valor_str) else 0
    
    # Verificar se contém três letras consecutivas
    letras_match = re.search(r'[a-zA-Z]{3}', valor_str)
    letras = letras_match.group() if letras_match else 0

    return numero, parenteses, letras

In [4]:
df_final = pd.DataFrame()

for file in files:
    competition = file.split('/')[-1].split('.')[0]
    classe_vela = competition.split(' - ')[1]
    competition = competition.split(' - ')[0]

    df_gold = pd.read_excel(file, sheet_name='Gold')
    df_silver = pd.read_excel(file, sheet_name='Silver')

    try:
        df_bronze = pd.read_excel(file, sheet_name='Bronze')
        dfs = [df_gold, df_silver, df_bronze]
    except:
        dfs = [df_gold, df_silver]


    for df in dfs:
        regatas = df.columns[6:-2]

        df = pd.melt(df, id_vars=['Rank', 'Fleet', 'Sailor', 'Country', 'Total', 'Net'],
                     value_vars=regatas, var_name="Regata", value_name="Pontuação Regata")
        
        df['Classe Vela'] = classe_vela
        df['Nome Competição'] = competition

        df_final = pd.concat([df_final, df])

In [5]:
df_final.head()

,Rank,Fleet,Sailor,Country,Total,Net,Regata,Pontuação Regata,Classe Vela,Nome Competição
0,1,Gold,Anne-Marie Rindom,DEN,88.0,40.0,Q1,1,Ilca 6,World Championship 2015
1,2,Gold,Marit Bouwmeester,NED,101.0,40.0,Q1,2,Ilca 6,World Championship 2015
2,3,Gold,Evi Van Acker,BEL,120.0,44.0,Q1,2,Ilca 6,World Championship 2015
3,4,Gold,Tuula Tenkanen,FIN,73.0,49.0,Q1,(12),Ilca 6,World Championship 2015
4,5,Gold,Josefin Olsson,SWE,96.0,53.0,Q1,(20),Ilca 6,World Championship 2015


In [6]:
column_order = ["ID Resultado", "ID Competidor", "Nome Competidor",
    "ID Competição", "Classe Vela", "Pontuação Regata", "Descarte",
    "Flotilha", "Posição Geral", "Punição", "Pontuação Total",
    "Nett", "Nome Competição", "Regata", "Nacionalidade"]

In [7]:
df_final['Pontuação Regata'] = df_final['Pontuação Regata'].apply(lambda x: str(x).replace('\n', ''))

df_final['Punição'] = df_final['Pontuação Regata'].apply(lambda x: formata_pontuacao(str(x))[2])
df_final['Descarte'] = df_final['Pontuação Regata'].apply(lambda x: formata_pontuacao(str(x))[1])
df_final['Pontuação Regata'] = df_final['Pontuação Regata'].apply(lambda x: formata_pontuacao(str(x))[0])

In [8]:
df_final.columns

Index(['Rank', 'Fleet', 'Sailor', 'Country', 'Total', 'Net', 'Regata',
       'Pontuação Regata', 'Classe Vela', 'Nome Competição', 'Punição',
       'Descarte'],
      dtype='object')

In [9]:
df_final.rename(columns={"Rank":"Posição Geral", 'Fleet': "Flotilha", 'Sailor': "Nome Competidor",
                         'Country': "Nacionalidade", "Total": "Pontuação Total",
                         "Net": "Nett"}, inplace=True)

df_final[["ID Resultado", "ID Competidor", "ID Competição"]] = '-'

df_final = df_final[column_order]

In [10]:
import unidecode
df_final["Nome Competidor"] = df_final["Nome Competidor"].apply(lambda x: unidecode.unidecode(x))
df_final["Nome Competidor"] = df_final["Nome Competidor"].apply(lambda x: str(x).upper())

In [11]:
df_final.head()

,ID Resultado,ID Competidor,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Descarte,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição,Regata,Nacionalidade
0,-,-,ANNE-MARIE RINDOM,-,Ilca 6,1.0,0,Gold,1,0,88.0,40.0,World Championship 2015,Q1,DEN
1,-,-,MARIT BOUWMEESTER,-,Ilca 6,2.0,0,Gold,2,0,101.0,40.0,World Championship 2015,Q1,NED
2,-,-,EVI VAN ACKER,-,Ilca 6,2.0,0,Gold,3,0,120.0,44.0,World Championship 2015,Q1,BEL
3,-,-,TUULA TENKANEN,-,Ilca 6,12.0,1,Gold,4,0,73.0,49.0,World Championship 2015,Q1,FIN
4,-,-,JOSEFIN OLSSON,-,Ilca 6,20.0,1,Gold,5,0,96.0,53.0,World Championship 2015,Q1,SWE


In [12]:
df_final.to_csv('../cleaned_data_2024/cluster17.csv', index=False)